# TIME LINE OF NEWS

**Aim :** Look into the spreading of news over websites during time

# Libraries

In [1]:
import os
import os.path
os.chdir("C:/Users/maell/Fake_News_Project")

import urllib.request
import zipfile

import pandas as pd
import numpy as np
import community

from scipy import *

from itertools import combinations
from collections import Counter

import matplotlib.pyplot as plt

import networkx as nx

# Functions

In [2]:
def open_data(fichier):
    data = pd.read_csv(fichier,sep = '\t',decimal = '.',index_col=0)
    return data

# Load the day we want to check

In [3]:
date='20170830'

In [5]:
file='all_data/all_data_'+date+'.csv'
all_data=open_data(file)
all_data.head()

,GlobalEventID,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras
0,573975381.0,2.016083e+13,2.017083e+13,1.0,thereflector.com,http://www.thereflector.com/home_scene/article...,6.0,-1.0,1409.0,1434.0,0.0,40.0,3641.0,3.001580,NaN,NaN
1,574286833.0,2.016083e+13,2.017083e+13,1.0,clarionledger.com,http://www.clarionledger.com/story/news/politi...,10.0,-1.0,2793.0,2869.0,1.0,50.0,5090.0,0.937082,NaN,NaN
2,573948616.0,2.016083e+13,2.017083e+13,1.0,thejournal.ie,http://www.thejournal.ie/public-services-card-...,36.0,-1.0,10662.0,10670.0,0.0,40.0,12130.0,-0.048591,NaN,NaN
3,574218333.0,2.016083e+13,2.017083e+13,1.0,cbslocal.com,http://minnesota.cbslocal.com/2017/08/29/mpls-...,6.0,-1.0,1299.0,1323.0,0.0,40.0,1408.0,-5.761317,NaN,NaN
4,685153804.0,2.017083e+13,2.017083e+13,1.0,enca.com,https://www.enca.com/south-africa/checkpoint-t...,2.0,285.0,-1.0,342.0,0.0,40.0,1045.0,-4.081633,NaN,NaN


## Check the events of this day

In [28]:
# Count number of publication
eventID=685157906
data_event=all_data[all_data.GlobalEventID==eventID]
print('Nb of publication :',len(data_event))
print(data_event.MentionIdentifier[613738 ])

Nb of publication : 1729
http://www.coloradodaily.com/business/ci_31258728/oskar-blues-halts-beer-production-longmont-can-water


In [5]:
c_event = Counter(all_data['GlobalEventID'])

In [ ]:
c_event.most_common()

## Create a reduce DataFrame containning only the event we care about

In [7]:
all_data_reduit=pd.DataFrame();
index_to_drop=[]

event_list=[685158048]

for event in event_list: #all_data['GlobalEventID'].unique(): un peu long -> on fait que sur les events qu'on veut tracer
    df_reduit=all_data[all_data['GlobalEventID']==event]
    df_reduit.sort_values(by=['MentionTimeDate'],inplace=True)
    for i in range (1,len(df_reduit)):
        if df_reduit['MentionTimeDate'].iloc[i-1]==df_reduit['MentionTimeDate'].iloc[i]:
            all_data_index=df_reduit.iloc[i].name
            index_to_drop.append(all_data_index) #we drop the "last" -> never drop the "0"
    all_data_reduit=all_data.drop(index_to_drop, inplace=False)
    

C:\Users\maell\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Create the graph of the event timeline

In [60]:
def graph_event_timeline(date,list_event):
    #ouverture du df
    file='all_data_'+date+'.csv'
    all_data=open_data(file)
    
    #correction du fd (suppression des lignes multiples (meme journal, meme event, meme horaire))
    all_data_reduit=pd.DataFrame();
    index_to_drop=[]
    for event in list_event: #all_data['GlobalEventID'].unique(): un peu long -> on fait que sur les events qu'on veut tracer
        df_reduit=all_data[all_data['GlobalEventID']==event]
        df_reduit.sort_values(by=['MentionTimeDate'],inplace=True)
        for i in range (1,len(df_reduit)):
            if df_reduit['MentionTimeDate'].iloc[i-1]==df_reduit['MentionTimeDate'].iloc[i] and df_reduit['MentionSourceName'].iloc[i-1]==df_reduit['MentionSourceName'].iloc[i]:
                all_data_index=df_reduit.iloc[i].name
                index_to_drop.append(all_data_index) #we drop the "last" -> never drop the "0"
    all_data_reduit=all_data.drop(index_to_drop, inplace=False)
        
    #creation du graph
    DG=nx.DiGraph()
    #pour chaque evenement que l'on souhaite regarder
    for event in list_event:
        #on récupère le df avec que cet evenement, et on le classe dans l'ordre d'apparition
        df=all_data_reduit[all_data_reduit['GlobalEventID']==event]
        
        #on créer les noeuds
        DG.add_nodes_from(df['MentionSourceName'])
        
        #on ajoute les liens
        df.sort_values(['MentionTimeDate'], inplace=True)
        list_time=list(df['MentionSourceName'])
     
        counter=1;
        c_node=0;
        for i in range (0,len(list_time)-1):
            DG.add_edge(list_time[i],list_time[i+1],timeline=counter)
            counter+=1
            try :
                DG.node[list_time[i]]['Time']
            except:
                DG.node[list_time[i]]['Time']=c_node
                c_node+=1
            
    return(DG)
###############################################################################################################################
def label_controversial_website(G): #OK
    fake_news_sources_df=pd.read_csv('Classification_Website_False.csv',sep = ';',decimal = '.')
    fake_news_sources_df.set_index('url',inplace=True)
    fake_news_sources=list(fake_news_sources_df.index.unique())
    for node in list(G.nodes):
        for site in fake_news_sources:
            if node==site:
                G.node[node]['Controversial Website']=fake_news_sources_df.loc[site].type
    return (G)

In [83]:
DG=graph_event_timeline(date,[685157906]) #685158048])
DG=label_controversial_website(DG)

C:\Users\maell\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\maell\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [84]:
nx.write_gexf(DG,'event_685157906_day_20170830.gexf')

## Study the event for interpretation

In [85]:
c=0
fake_news_sources_df=pd.read_csv('Classification_Website_False.csv',sep = ';',decimal = '.')
fake_news_sources_df.set_index('url',inplace=True)
fake_news_sources=list(fake_news_sources_df.index.unique())

for site in list(all_data_reduit[all_data_reduit['GlobalEventID']==685157906].MentionSourceName):
    if site in fake_news_sources:
        print(site)
        c+=1
        
print(c)

twitchy.com
washingtonexaminer.com
dailytelegraph.com.au
thedailybeast.com
patriotpost.us
theblaze.com
politicususa.com
politicususa.com
westernjournalism.com
thinkprogress.org
express.co.uk
washingtonexaminer.com
12


In [70]:
all_data[all_data['GlobalEventID']==685158928].MentionIdentifier[16144]

'http://k99country.iheart.com/content/2017-08-29-president-trump-arrives-in-coastal-bend-to-survey-hurricane-harvey-damage/'